# Introduction to Geospatial AI

# Intro
Welcome to this workship about geospatial AI. In this workshop you will try to detect building from aerial images. This is done in three steps;

1. create training data.
2. training machine learning models.
3. evaluating the trained models and predicting where buildings are in new images. 

# Task 0

First we need to set up the environment properly. In order to do that follow the steps under;

1. Create copy of notebook. Do this by saving the notebook. Shortcut is `Ctrl + s` on Windows or `Cmd + s` on Mac.
2. Fix GPU runtime environment. In the top right corner choose `Change runtime type` and select `T4 GPU`.
3. Insert secrets.

When all the items on the list above are completed we need to clone the git repo we are working with and adding it to the path in addition to installing some dependencies. To do this simply run the two cells below.


In [ ]:
!git clone https://github.com/kartAI/kartAI.git

!pip install focal_loss
!pip install azure-storage-blob
!pip install rasterio
!pip install rasterstats

In [ ]:
import sys
sys.path.insert(0,'/content/kartAI')

# Task 1 - Create training data
In the first task we create the training data by 

https://geojson.io/

In [ ]:
from kartAI.kartai.tools.create_training_data import create_training_data

create_training_data(training_dataset_name="test", config_file_path="kartAI/config/dataset/bygg.json", eager_load=True,
                         confidence_threshold=None, eval_model_checkpoint=None,
                         region=None, x_min=618296.0, x_max=623495.0, y_min=6668145.0, y_max=6672133.0,
                         num_processes=None)

# Task 2 - Train a machine learning model

In [ ]:
from kartAI.kartai.tools.train import train

train_args = {
        "features": 32,
        "depth": 4,
        "optimizer": "RMSprop",
        "batch_size": 8,
        "model": "unet",
        "loss": "binary_crossentropy",
        "activation": "relu",
        "epochs": 1
}


train(checkpoint_name="some_checkpoint",
      dataset_name=["test"],
      input_generator_config_path="kartAI/config/ml_input_generator/ortofoto.json",
      save_model=False,
      train_args=train_args,
      checkpoint_to_finetune=False
)


# Task 3 - Evaluation and Inference

In [ ]:
import os
import json
from kartAI.env import get_env_variable
from kartAI.kartai.tools.predict import predict_and_evaluate

created_datasets_dir = os.path.join(get_env_variable(
    'created_datasets_directory'), "test")

checkpoint_path = os.path.join(get_env_variable(
    'trained_models_directory'), 'some_checkpoint.h5')

with open("kartAI/config/ml_input_generator/ortofoto.json", encoding="utf8") as config:
    datagenerator_config = json.load(config)

predict_and_evaluate(
    created_datasets_dir,
    datagenerator_config,
    "some_checkpoint",
    True,
    True
)